# Context Engineering – Retrieval model integration
We'll configure DSPy with a simple custom retriever and your documents.


In [10]:
from dotenv import load_dotenv
load_dotenv()

import dspy

# Configure LLM (assumes OPENAI_API_KEY set)
llm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm=llm)
print('LLM configured:', llm.model)


LLM configured: openai/gpt-4o-mini


## 1) Baseline vs Context-Augmented Prompting
Compare answers with and without added context.


In [11]:
question = "What are the key benefits of DSPy for LLM applications?"

baseline = dspy.Predict('question -> answer')
base_resp = baseline(question=question)
print('Baseline answer:\n', base_resp.answer)

context = (
    "DSPy automates prompt engineering, supports program compilation, "
    "and provides optimizers for few-shot, instruction tuning, and finetuning."
)
with_ctx = dspy.Predict('question, context -> answer')
ctx_resp = with_ctx(question=question, context=context)
print('\nWith context:\n', ctx_resp.answer)


Baseline answer:
 DSPy offers several key benefits for LLM (Large Language Model) applications, including:

1. **Ease of Use**: DSPy provides a user-friendly interface that simplifies the process of building and deploying models, making it accessible for users with varying levels of expertise.

2. **Integration with LLMs**: It allows seamless integration with existing LLMs, enabling users to leverage the power of these models without needing extensive modifications.

3. **Customizability**: Users can easily customize their models to fit specific use cases, allowing for tailored solutions that meet unique business needs.

4. **Scalability**: DSPy is designed to handle large datasets and complex models, ensuring that applications can scale as needed without performance degradation.

5. **Rapid Prototyping**: The platform supports quick iterations and prototyping, allowing developers to test and refine their models efficiently.

6. **Robustness**: DSPy includes features that enhance the r

## 2) Simple Retrieval-Grounded Context
Retrieve snippets from a tiny in-memory corpus and inject them as context.


In [13]:
from difflib import SequenceMatcher

corpus = [
    "DSPy composes LLM programs with explicit signatures and modules.",
    "It provides optimizers like LabeledFewShot, BootstrapFewShot, and MIPROv2.",
    "DSPy integrates with OpenAI and can use tools like Tavily for ReAct.",
]

def retrieve(query: str, k: int = 2):
    scored = []
    for doc in corpus:
        score = SequenceMatcher(a=query.lower(), b=doc.lower()).ratio()
        scored.append((score, doc))
    scored.sort(reverse=True)
    return [doc for _, doc in scored[:k]]

query = "How does DSPy help structure LLM apps?"
snippets = retrieve(query)
print('Retrieved context:')
for s in snippets:
    print('-', s)

rag = dspy.Predict('question, context -> answer')
ctx = "\n".join(snippets)
resp = rag(question=query, context=ctx)
print('\nAnswer with retrieved context:\n', resp.answer)


Retrieved context:
- It provides optimizers like LabeledFewShot, BootstrapFewShot, and MIPROv2.
- DSPy composes LLM programs with explicit signatures and modules.

Answer with retrieved context:
 DSPy helps structure LLM apps by providing optimizers such as LabeledFewShot, BootstrapFewShot, and MIPROv2, which enhance the performance of the models. Additionally, it composes LLM programs with explicit signatures and modules, allowing for better organization and clarity in the development of applications.


## 3) Custom Retrieval Model 
A keyword-based retriever integrated into DSPy, used for RAG-style grounding.


In [14]:
# Your original snippet placed here (unchanged)

documents = [
    "The core idea of DSPy is to separate program logic from the specifics of prompts.",
    "DSPy's optimizers, called teleprompters, can automatically tune prompts using examples.",
    "A retrieval model (RM) is a tool used to find relevant passages from a large knowledge source.",
    "ReAct is a DSPy module that creates an agent capable of reasoning and using tools in a loop.",
    "To build a RAG system, you need a retriever for fetching context and a generator for synthesizing answers."
]

class SimpleRM(dspy.Retrieve):
    def __init__(self, docs, k=3):
        super().__init__(k=k)
        self.documents = [dspy.Example(long_text=doc) for doc in docs]

    def forward(self, query_or_queries, k=None):
        if k is None:
            k = self.k
        # Assuming query_or_queries is a single query string based on how it's used in the pipeline
        query = query_or_queries.lower()
        keywords = query.split()
        found_docs = [
            doc for doc in self.documents
            if any(keyword in doc.long_text.lower() for keyword in keywords)
        ]
        results = found_docs[:k]
        return dspy.Prediction(passages=results)

retrieval_model = SimpleRM(documents, k=3)

dspy.configure(lm=llm, rm=retrieval_model)
print("✅ DSPy environment configured with GPT-4o-mini and a retrieval model.")


✅ DSPy environment configured with GPT-4o-mini and a retrieval model.


## 4) Advanced Context Pipeline
Query transform → retrieve → context refinement → answer synthesis.


In [20]:
# Query Transformation
class GenerateSearchQuery(dspy.Signature):
    """Generate a clear, concise search query from a user's question."""
    question = dspy.InputField()
    search_query = dspy.OutputField()

# Context Refinement
class FilterRelevantPassages(dspy.Signature):
    """From a list of passages, select only those that help answer the question."""
    question = dspy.InputField()
    passages = dspy.InputField(desc="A list of passages, each separated by '---'.")
    relevant_passages = dspy.OutputField(desc="A list of the most relevant passages, each separated by '---'.")

# Answer Synthesis
class SynthesizeFinalAnswer(dspy.Signature):
    """Answer the user's question faithfully using the provided context."""
    context = dspy.InputField(desc="Relevant facts and information.")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="A comprehensive answer to the question, citing the context.")

class AdvancedContextPipeline(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_query = dspy.Predict(GenerateSearchQuery)
        self.retrieve = dspy.Retrieve(k=3)
        self.filter_passages = dspy.ChainOfThought(FilterRelevantPassages)
        self.generate_answer = dspy.ChainOfThought(SynthesizeFinalAnswer)

    def forward(self, question):
        generated_query = self.generate_query(question=question).search_query
        retrieved = self.retrieve(generated_query).passages
        # Passages may be dspy.Example objects; convert to strings if needed
        passages_list = [getattr(p, 'long_text', str(p)) for p in retrieved]
        passages_str = "\n---\n".join(passages_list)
        refined_context_str = self.filter_passages(question=question, passages=passages_str).relevant_passages
        refined_context = refined_context_str.split("\n---\n") if refined_context_str else []
        final_prediction = self.generate_answer(context="\n".join(refined_context), question=question)
        return dspy.Prediction(answer=final_prediction.answer)

print("✅ AdvancedContextPipeline defined.")


✅ AdvancedContextPipeline defined.


In [21]:
# Revised pipeline to accept explicit retriever and normalize passages
class AdvancedContextPipeline(dspy.Module):
    def __init__(self, retriever=None):
        super().__init__()
        self.generate_query = dspy.Predict(GenerateSearchQuery)
        # Use provided retriever (e.g., SimpleRM) or default DSPy retriever
        self.retrieve = retriever or dspy.Retrieve(k=3)
        self.filter_passages = dspy.ChainOfThought(FilterRelevantPassages)
        self.generate_answer = dspy.ChainOfThought(SynthesizeFinalAnswer)

    def forward(self, question):
        generated_query = self.generate_query(question=question).search_query
        retrieved = self.retrieve(generated_query).passages

        # Normalize to a list of strings
        passages_list = []
        if isinstance(retrieved, str):
            passages_list = [retrieved]
        else:
            for p in retrieved:
                # p may be dspy.Example or a plain string
                text = getattr(p, 'long_text', None)
                passages_list.append(text if isinstance(text, str) else str(p))

        passages_str = "\n---\n".join(passages_list)
        refined_context_str = self.filter_passages(question=question, passages=passages_str).relevant_passages
        refined_context = refined_context_str.split("\n---\n") if refined_context_str else []
        final_prediction = self.generate_answer(context="\n".join(refined_context), question=question)
        return dspy.Prediction(answer=final_prediction.answer)

print("✅ Revised AdvancedContextPipeline defined.")


✅ Revised AdvancedContextPipeline defined.


## 5) Optimizing the Pipeline with BootstrapFewShot
We optimize `AdvancedContextPipeline` using a small train set and a simple validation metric.


In [22]:
from dspy.teleprompt import BootstrapFewShot

# Train data ---
train_data = [
    dspy.Example(
        question="How are prompts tuned in DSPy?",
        answer="DSPy's optimizers, called teleprompters, can automatically tune prompts using examples."
    ).with_inputs("question"),
    dspy.Example(
        question="What is the ReAct module?",
        answer="ReAct is a DSPy module that creates an agent capable of reasoning and using tools in a loop."
    ).with_inputs("question"),
]

# Validation metric ---
def validate_context_and_answer(gold, pred, trace=None):
    if isinstance(pred.answer, str) and isinstance(gold.answer, str):
        return gold.answer.lower() in pred.answer.lower()
    return False

# Optimizer ---
optimizer = BootstrapFewShot(metric=validate_context_and_answer, max_bootstrapped_demos=2)

# Compile ---
unoptimized_pipeline = AdvancedContextPipeline()
optimized_pipeline = optimizer.compile(unoptimized_pipeline, trainset=train_data)

print("✅ Pipeline optimization complete.")


  0%|          | 0/2 [00:00<?, ?it/s]2025/09/05 06:00:47 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example Example({'question': 'How are prompts tuned in DSPy?', 'answer': "DSPy's optimizers, called teleprompters, can automatically tune prompts using examples."}) (input_keys={'question'}) with <function validate_context_and_answer at 0x11e4982c0> due to 'str' object has no attribute 'long_text'.
2025/09/05 06:00:47 ERROR dspy.teleprompt.bootstrap: Failed to run or to evaluate example Example({'question': 'What is the ReAct module?', 'answer': 'ReAct is a DSPy module that creates an agent capable of reasoning and using tools in a loop.'}) (input_keys={'question'}) with <function validate_context_and_answer at 0x11e4982c0> due to 'str' object has no attribute 'long_text'.
100%|██████████| 2/2 [00:00<00:00, 559.17it/s]

Bootstrapped 0 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.
✅ Pipeline optimization complete.


In [23]:
# Recreate optimized pipeline using the revised class and explicit retriever
try:
    from dspy.teleprompt import BootstrapFewShot
    optimizer = BootstrapFewShot(metric=validate_context_and_answer, max_bootstrapped_demos=2)
    unoptimized_pipeline = AdvancedContextPipeline(retriever=retrieval_model)
    optimized_pipeline = optimizer.compile(unoptimized_pipeline, trainset=train_data)
    pipeline = optimized_pipeline
    print('✅ Recompiled optimized pipeline with explicit retriever')
except Exception as e:
    pipeline = AdvancedContextPipeline(retriever=retrieval_model)
    print(f'⚠️ Using unoptimized pipeline due to: {e}')

# Run
user_question = "Explain the role of DSPy's teleprompters."
final_answer = pipeline(user_question)

print("\n" + "="*50)
print(f"Original Question: {user_question}")
print(f"Final Synthesized Answer: {getattr(final_answer, 'answer', final_answer)}")
print("="*50)

try:
    llm.inspect_history(n=1)
except Exception as e:
    print(f"(History not available: {e})")


100%|██████████| 2/2 [00:10<00:00,  5.09s/it]


Bootstrapped 0 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.
✅ Recompiled optimized pipeline with explicit retriever

Original Question: Explain the role of DSPy's teleprompters.
Final Synthesized Answer: The role of DSPy's teleprompters is to automatically tune prompts using examples, thereby optimizing their effectiveness for specific tasks and improving system performance.




[2025-09-05T06:01:18.581417]

System message:

Your input fields are:
1. `context` (str): Relevant facts and information.
2. `question` (str):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str): A comprehensive answer to the question, citing the context.
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Answer the us

In [29]:
# Run the optimized pipeline safely
user_question = "Explain the role of DSPy's teleprompters."

# Prefer optimized pipeline if available; else fallback
pipeline = globals().get('optimized_pipeline') or globals().get('unoptimized_pipeline') or AdvancedContextPipeline()

final_answer = pipeline(user_question)

print("\n" + "="*50)
print(f"Original Question: {user_question}")
print(f"Final Synthesized Answer: {getattr(final_answer, 'answer', final_answer)}")
print("="*50)

# Inspect most recent LLM interaction
try:
    llm.inspect_history(n=1)
except Exception as e:
    print(f"(History not available: {e})")



Original Question: Explain the role of DSPy's teleprompters.
Final Synthesized Answer: The role of DSPy's teleprompters is to automatically tune prompts using examples, thereby optimizing their effectiveness for specific tasks and improving system performance.




[2025-09-05T06:12:52.488685]

System message:

Your input fields are:
1. `context` (str): Relevant facts and information.
2. `question` (str):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str): A comprehensive answer to the question, citing the context.
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Answer the user's question faithfully using the provided context.


User message:

This is an example of the task, though some input or output fields are not 

Robust site-restricted search (handles Tavily return shapes)
Normalize Tavily results (dict/list/strings), filter to `https://dspy.ai/`, summarize to 3 lines, and augment the answer.


In [31]:
# Robust normalization + strict filter + summary + augmented answer
import os
from tavily import TavilyClient

client = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))

user_question = "What does DSPy mean by modules and optimizers?"
raw = client.search(f"site:dspy.ai {user_question}", max_results=8)

# Normalize raw into a list of dicts with url/title/content
norm = []
if isinstance(raw, dict) and 'results' in raw:
    items = raw['results']
else:
    items = raw if isinstance(raw, list) else [raw]

for item in items:
    if isinstance(item, dict):
        url = str(item.get('url', ''))
        title = str(item.get('title', ''))
        content = str(item.get('content', ''))
    else:
        # item may be a string; treat as content
        url, title, content = '', '', str(item)
    norm.append({'url': url, 'title': title, 'content': content})

only_official = [r for r in norm if r['url'].startswith('https://dspy.ai/') or 'dspy.ai' in r['url']]

materials = []
for r in only_official[:5]:
    blob = (r['title'] + ' ' + (r['content'] or r['url'])).strip()
    materials.append(blob)
material = "\n\n".join(materials) if materials else "DSPy uses modules to express behavior and optimizers to tune them."

summary = dspy.Predict('material -> summary_3_lines')(material=material).summary_3_lines
aug_answer = dspy.ChainOfThought('question, external_context -> answer')
final = aug_answer(question=user_question, external_context=summary)

print('Summary (3 lines):\n', summary)
print('\nAnswer:\n', final.answer)



Summary (3 lines):
 DSPy is a programming model that allows users to describe AI behavior through code rather than strings, utilizing modules and optimizers for enhanced functionality. The DSPy compiler adapts to changes in data and control flow, optimizing prompts and weights for specific pipelines. Its ecosystem includes key components like LMs, signatures, modules, and optimizers, facilitating efficient program evaluation and parallelization.

Answer:
 In DSPy, "modules" are components that encapsulate specific functionalities for building AI behaviors, while "optimizers" are mechanisms that enhance model performance by adjusting parameters and adapting to changes in data and control flow.


As we can inspect in the example, adding additinal or advance context only required cofiguring signature and triggering one of the modules with the additional context . In traditional format , We have to prompt engineer this additional context separately